In [1]:
import numpy as np

## Q1

In [5]:
def pretty_print(state: str):
    size = int(np.sqrt(len(state)))
    for i in range(size):
        print("  ".join(state[size*i:size*(i+1)]))

state = 'XXOOX XOO'
pretty_print(state)
int(np.sqrt(len(state)))

X  X  O
O  X   
X  O  O


3

In [18]:
k = 5
n = 100
x, y = 9,2
print(np.max([x-(k-1), 0]), np.min([x+(k-1), n-1]))

5 13


In [20]:
np.random.choice([1,2,3])

2

In [9]:
state = np.array([
    [1,1,-1],
    [0,0,0],
    [1,1,1]
])

s = state.tolist()


In [4]:
"""
###############
##Tic-Tac-Toe##
###############

幻想用一个 program code n board size, k c.t.s. win 是愚蠢的。一个 program 的 extension 应当有一定的局限性才对。

Define
    state = [[1,2,3], [4,5,6], index]
where
    - The first and second lists are the location of the X's and O's respectively;
    - Index refer to the index of a large array holding the value of the state.
Note that without eliminating the cases when there exist both XXX and OOO, the total possible states is 6046.
The number is acceptable as the dict later on is easier to construct compared with the true number 5478.
    (len([]) = 0, len([])=0) -> comb(9,0)*comb(9,0)
    (len([]) = 1, len([])=0) -> comb(9,1),comb(8,0)
    (len([]) = 1, len([])=0) -> comb(9,1)*comb(8,1)
    ...
    (len([]) = 5, len([])=4) -> comb(9,5),comb(4,4)
Here, the order of index of the locations in the board does not matter. They are put into the box of X and O.
Hence, we can define the dict as
    state_values[(tuple(X_moves), tuple(O_moves))] -> value

Notice that each state can be reached even it is a terminal state. The only difference is the order to reach the state.
Hence, define the index of each state using the X-moves and O-moves as



Define the index of the location as follows:
    2 9 4
    7 5 3
    6 1 8
Then define the index of the location of the X's and the O's separately 2 arrays as, e.g.,
    1 2 3
    4 5 6
As long as the sum of the index = 15, then win.
Notice that we cannot have 2


    Value_table = -np.ones(6046)

TODO: As the loss value is 0 and win value is 1, the init value does not necessarily need to be None?
Otherwise it keeps on warning that None - None


English-Chines Dictionary:
    Bit - 落子

"""
from typing import List

import numpy as np
import itertools
from math import comb

magic_square = np.array([2, 9, 4, 7, 5, 3, 6, 1, 8])


def any_n_sum_to_k(moves: List[int], quiet=True):
    """
    Return True if there exist one XXX or OOO in the board. False if not.
    If set quiet = False, print ONLY one of the XXX or OOO index of location in the board.
    """
    result = False
    for sub_moves in itertools.combinations(moves, 3):
        if np.sum(sub_moves) == 15:
            if not quiet:
                print(sub_moves)

            result = True

    return result


def show_state(state, quiet=False):
    """
    Pretty_print.
    """
    if not quiet:
        X_moves = state[0].copy()
        O_moves = state[1].copy()
        dig_board = list(magic_square)
        board = list(magic_square.astype(str))

        for X in X_moves:
            X_i = dig_board.index(X)
            board[X_i] = 'X'
        for O in O_moves:
            O_i = dig_board.index(O)
            board[O_i] = 'O'

        for i in range(3):
            print("  ".join(board[3 * i: 3 * (i + 1)]))


def init_value_table(quiet=True):
    """
    Init the value table.
    """
    value_table = {}

    board = list(range(1, 9 + 1))
    n = len(board)
    for k in range(1, 5 + 1):
        if not quiet:
            print(
                print("comb(9,%d)*comb(%d,%d)" % (k - 1, n - (k - 1), k - 1), comb(n, k - 1), comb(n - (k - 1), k - 1)))
        for X_moves in itertools.combinations(board, k - 1):
            for O_moves in itertools.combinations(list(set(board) - set(X_moves)), k - 1):
                value_table[(tuple(sorted(X_moves)), tuple(sorted(O_moves)))] = None

        if not quiet:
            print("comb(9,%d),comb(%d,%d)" % (k, n - k, k - 1), comb(n, k), comb(n - k, k - 1))
        for X_moves in itertools.combinations(board, k):
            for O_moves in itertools.combinations(list(set(board) - set(X_moves)), k - 1):
                value_table[(tuple(sorted(X_moves)), tuple(sorted(O_moves)))] = None

    return value_table


value_table = init_value_table()
init_state = [[], []]


def value(state):
    X_moves = tuple(sorted(state[0].copy()))
    O_moves = tuple(sorted(state[1].copy()))

    return value_table[(X_moves, O_moves)]


def set_value(state, value):
    X_moves = tuple(sorted(state[0].copy()))
    O_moves = tuple(sorted(state[1].copy()))

    value_table[(X_moves, O_moves)] = value


def next_state(player, state, move):
    """
    Return the new state after making the indicated move by the indicated player.
    Here, it is important to keep the terminal state with value 1 and 0 instead of 1.0 and 0.0, as the type is used to
    determine whether it is a terminal state or not.

    # This actually does not mater as both state will terminate before happens.
    By checking the X-moves first, this function make sure that the next state of (2,9,4), (5,7,3) is valued by 1.
    However, (2,9,1,4), (5,7,3) is valued at 1 as well, even though it has lost.
    :param player: 'X' or 'O'
    """
    X_moves = state[0].copy()
    O_moves = state[1].copy()
    if player == 'X':
        X_moves.append(move)
    elif player == "O":
        O_moves.append(move)

    next_state = [X_moves, O_moves]
    if value(next_state) == None:
        if any_n_sum_to_k(X_moves):
            set_value(next_state, 1)
        elif any_n_sum_to_k(O_moves):
            set_value(next_state, 0)
        elif len(X_moves) + len(O_moves) == 9:
            set_value(next_state, 0)
        else:
            set_value(next_state, 0.5)

    return next_state


def terminal_state_p(state):
    """
    If value of state is int, then it is terminal; otherwise, it is not a terminal
    """
    return isinstance(value(state), int)


alpha = 0.5
epsilon = 0.01


def possible_moves(state):
    """
    Return the possible moves. Here it is denoted as blanks.
    """

    blanks = []
    for i in range(1, 9 + 1):
        X_moves = state[0].copy()
        O_moves = state[1].copy()
        if i not in X_moves and i not in O_moves:
            blanks.append(i)

    return blanks


def random_move(state):
    """
    Return one of the unplaced locations, and selected at random
    """

    blanks = possible_moves(state)

    return np.random.choice(blanks)


def greedy_move(player, state):
    """
    Return the move which gives the highest valued position when played.
    """
    # Map the blanks and values using their index
    blanks = possible_moves(state)
    values = [value(next_state(player, state, move)) for move in blanks]

    # Obtain one action index of the max estimated action value.
    max_value = np.max(values)

    # Obtain all actions index of the max estimated action value.
    greedy_index_list = [index for index, value in enumerate(values) if value == max_value]

    greedy_index = np.random.choice(greedy_index_list)

    greedy_move = blanks[greedy_index]

    return greedy_move


# Now here is the main function
initial_state = [[], []]
value_table[((), ())] = 0.5


def game(quiet=False):
    """
    Plays 1 game against the random player. Also learns and prints.
    """
    state = initial_state
    show_state(state, quiet)

    while True:
        new_state = next_state('X', state, random_move(state))
        exploratory = np.random.binomial(n=1, p=epsilon)

        if terminal_state_p(new_state):
            show_state(new_state, quiet)

            update(state, new_state, quiet)

            return value(new_state)

        if exploratory:
            move = random_move(new_state)
        else:
            move = greedy_move('O', new_state)

        new_state = next_state('O', new_state, move)

        # TODO: Why I need a unless here?
        if not exploratory:
            update(state, new_state, quiet)
        show_state(new_state, quiet)

        if terminal_state_p(new_state):
            return value(new_state)

        state = new_state


def update(state, new_state, quiet=False):
    """
    This is the learning rule.
    """
    set_value(state, value(state) + alpha * (value(new_state) - value(state)))
    if not quiet:
        print(state, value(state))


def run():
    for _ in range(40):
        v = 0
        for _ in range(100):
            v += game()

        print(v / 100.0)


def runs(num_runs, num_bins, bin_size):
    array = np.zeros(num_bins)
    pass


if __name__ == "__main__":
    run()


2  9  4
7  5  3
6  1  8
[[], []] 0.5
O  9  4
X  5  3
6  1  8
[[7], [2]] 0.5
O  9  O
X  5  3
X  1  8
[[7, 6], [2, 4]] 0.5
O  X  O
X  5  3
X  O  8
[[7, 6, 9], [2, 4, 1]] 0.5
O  X  O
X  5  O
X  O  X
O  X  O
X  X  O
X  O  X
[[7, 6, 9, 8], [2, 4, 1, 3]] 0.25
2  9  4
7  5  3
6  1  8
[[], []] 0.5
2  9  O
7  5  3
X  1  8
[[6], [4]] 0.5
2  9  O
7  O  3
X  X  8
[[6, 1], [4, 5]] 0.5
O  9  O
7  O  X
X  X  8
O  9  O
7  O  X
X  X  X
[[6, 1, 3], [4, 5, 2]] 0.75
2  9  4
7  5  3
6  1  8
[[], []] 0.5
X  9  4
7  5  O
6  1  8
[[2], [3]] 0.5
X  X  4
O  5  O
6  1  8
[[2, 9], [3, 7]] 0.5
X  X  4
O  5  O
6  X  O
X  X  4
O  X  O
6  X  O
[[2, 9, 1], [3, 7, 8]] 0.75
2  9  4
7  5  3
6  1  8
[[], []] 0.5
O  9  X
7  5  3
6  1  8
[[4], [2]] 0.5
O  9  X
7  X  3
O  1  8
[[4, 5], [2, 6]] 0.5
O  9  X
7  X  X
O  O  8
[[4, 5, 3], [2, 6, 1]] 0.25
O  X  X
7  X  X
O  O  O
2  9  4
7  5  3
6  1  8
[[], []] 0.5
2  9  4
7  O  X
6  1  8
[[3], [5]] 0.5
2  9  4
X  O  X
O  1  8
[[3, 7], [5, 6]] 0.5
X  9  4
X  O  X
O  O  8
[[3, 7, 2]

In [18]:
value_table

{((), ()): 0.98564421473993,
 ((1,), ()): 0.5,
 ((2,), ()): 0.5,
 ((3,), ()): 0.5,
 ((4,), ()): 0.5,
 ((5,), ()): 0.5,
 ((6,), ()): 0.5,
 ((7,), ()): 0.5,
 ((8,), ()): 0.5,
 ((9,), ()): 0.5,
 ((1,), (2,)): 0.9833107056581407,
 ((1,), (3,)): 0.6491158381625927,
 ((1,), (4,)): 0.5,
 ((1,), (5,)): 0.5,
 ((1,), (6,)): 0.5,
 ((1,), (7,)): 0.5,
 ((1,), (8,)): 0.5,
 ((1,), (9,)): 0.5,
 ((2,), (1,)): 0.5,
 ((2,), (3,)): 0.5,
 ((2,), (4,)): 0.9566414735636901,
 ((2,), (5,)): 0.5,
 ((2,), (6,)): 0.5,
 ((2,), (7,)): 0.5,
 ((2,), (8,)): 0.5,
 ((2,), (9,)): 0.5,
 ((3,), (1,)): 0.8563773571200611,
 ((3,), (2,)): 0.8089609189980774,
 ((3,), (4,)): 0.5,
 ((3,), (5,)): 0.5,
 ((3,), (6,)): 0.5,
 ((3,), (7,)): 0.5,
 ((3,), (8,)): 0.5,
 ((3,), (9,)): 0.5,
 ((4,), (1,)): 0.7184196829895982,
 ((4,), (2,)): 0.5,
 ((4,), (3,)): 0.5,
 ((4,), (5,)): 0.5,
 ((4,), (6,)): 0.5,
 ((4,), (7,)): 0.5,
 ((4,), (8,)): 0.60003662109375,
 ((4,), (9,)): 0.9499634237547512,
 ((5,), (1,)): 0.5,
 ((5,), (2,)): 0.5,
 ((5,), (3,

In [14]:
k = 5
quiet = True
board = list(range(1,9+1))
n = len(board)
for k in range(1, 5+1):
    if not quiet:
        print("comb(9,%d),comb(%d,%d)" % (k, n-k, k-1), comb(n, k), comb(n-k, k-1))
    for X_moves in itertools.combinations(board, k):
        for O_moves in itertools.combinations(list(set(board) - set(X_moves)), k - 1):
            print((X_moves, sorted(O_moves)))

((1,), [])
((2,), [])
((3,), [])
((4,), [])
((5,), [])
((6,), [])
((7,), [])
((8,), [])
((9,), [])
((1, 2), [3])
((1, 2), [4])
((1, 2), [5])
((1, 2), [6])
((1, 2), [7])
((1, 2), [8])
((1, 2), [9])
((1, 3), [2])
((1, 3), [4])
((1, 3), [5])
((1, 3), [6])
((1, 3), [7])
((1, 3), [8])
((1, 3), [9])
((1, 4), [2])
((1, 4), [3])
((1, 4), [5])
((1, 4), [6])
((1, 4), [7])
((1, 4), [8])
((1, 4), [9])
((1, 5), [2])
((1, 5), [3])
((1, 5), [4])
((1, 5), [6])
((1, 5), [7])
((1, 5), [8])
((1, 5), [9])
((1, 6), [2])
((1, 6), [3])
((1, 6), [4])
((1, 6), [5])
((1, 6), [7])
((1, 6), [8])
((1, 6), [9])
((1, 7), [2])
((1, 7), [3])
((1, 7), [4])
((1, 7), [5])
((1, 7), [6])
((1, 7), [8])
((1, 7), [9])
((1, 8), [2])
((1, 8), [3])
((1, 8), [4])
((1, 8), [5])
((1, 8), [6])
((1, 8), [7])
((1, 8), [9])
((1, 9), [2])
((1, 9), [3])
((1, 9), [4])
((1, 9), [5])
((1, 9), [6])
((1, 9), [7])
((1, 9), [8])
((2, 3), [1])
((2, 3), [4])
((2, 3), [5])
((2, 3), [6])
((2, 3), [7])
((2, 3), [8])
((2, 3), [9])
((2, 4), [1])
((2, 

In [216]:
quiet = False

state = initial_state
show_state(state, False)

new_state = next_state('X', state, random_move(state))
exploratory = np.random.binomial(n=1, p=epsilon)

if exploratory:
    move = random_move(new_state)
else:
    move = greedy_move('O', new_state)

move
new_state
new_state = next_state('O', new_state, move)
print(new_state)
show_state(new_state, quiet)

2  9  4
7  5  3
6  1  8
[[7], [1]]
2  9  4
X  5  3
6  O  8


In [ ]:
keys = {}
for key in value_table.keys():
    if value_table[key] != None:
        keys[key] = value_table[key]

keys

In [185]:
s = [[6,9,4], [3,2]]
state = s
# (sorted((2,9,4)),(3,5)) in v.keys()
# v.keys()
random_move(state)

8

In [194]:
player = 'O'
blanks = possible_moves(state)
move_values = [get_value(next_state(player, state, move)) for move in blanks]
print(blanks)
print(move_values)


[1, 5, 7, 8]
[0.5, 0.5, 0.5, 0.5]


In [198]:
print(greedy_move('O', state))
show_state(state)

7
O  X  X
7  5  O
X  1  8


In [155]:
print(get_value(next_state('O', s, 7)))
show_state(next_state('O', s, 7))

0
2  X  X
O  O  O
6  X  8


In [10]:
state = [[1],[],0]
X_moves = state[0]
O_moves = state[1]
# board = list(magic_square.astype(str).copy())
board = list(magic_square.astype(str))

for X in X_moves:
    X_i = board.index(X)
    board[X_i] = 'X'
for O in O_moves:
    O_i = board.index(O)
    board[O_i] = 'O'

for i in range(3):
    print("  ".join(board[3 * i: 3 * (i + 1)]))

ValueError: 1 is not in list

In [25]:
state = [[1,5,4,3],[2,9,6,7],0]
show_state(state)
512*512

O  O  X
O  X  X
O  X  8


262144

In [22]:
from math import comb


9

In [28]:
2**9

512

In [88]:
sum_comb = 0
for k in range(1,5+1):
    print("comb(9,%d)*comb(%d,%d)" % (k-1,9-k+1,k-1), comb(9,k-1), comb(9-(k-1),k-1))
    sum_comb += comb(9,k-1)*comb(9-(k-1),k-1)
    print("comb(9,%d),comb(%d,%d)"% (k,9-k,k-1), comb(9,k),comb(9-k,k-1))
    sum_comb += comb(9,k)*comb(9-k,k-1)

sum_comb

comb(9,0)*comb(9,0) 1 1
comb(9,1),comb(8,0) 9 1
comb(9,1)*comb(8,1) 9 8
comb(9,2),comb(7,1) 36 7
comb(9,2)*comb(7,2) 36 21
comb(9,3),comb(6,2) 84 15
comb(9,3)*comb(6,3) 84 20
comb(9,4),comb(5,3) 126 10
comb(9,4)*comb(5,4) 126 5
comb(9,5),comb(4,4) 126 1


6046

In [75]:
import itertools

v = {}

stuff = [1, 2, 3, 4, 5, 6, 7, 8, 9]
for k in range(1, len(stuff)+1):
    print(print("comb(9,%d)*comb(%d,%d)" % (k-1,9-k+1,k-1), comb(9,k-1), comb(9-(k-1),k-1)))
    for subset in itertools.combinations(stuff, k-1):
        for a_subset in itertools.combinations(list(set(stuff)-set(subset)), k-1):
            # print('\t',subset, a_subset)
            v[(subset, a_subset)] = None

    print("comb(9,%d),comb(%d,%d)"% (k,9-k,k-1), comb(9,k),comb(9-k,k-1))
    for subset in itertools.combinations(stuff, k):
        for a_subset in itertools.combinations(list(set(stuff)-set(subset)), k-1):
            v[(subset, a_subset)] = None



comb(9,0)*comb(9,0) 1 1
None
comb(9,1),comb(8,0) 9 1
comb(9,1)*comb(8,1) 9 8
None
comb(9,2),comb(7,1) 36 7
comb(9,2)*comb(7,2) 36 21
None
comb(9,3),comb(6,2) 84 15
comb(9,3)*comb(6,3) 84 20
None
comb(9,4),comb(5,3) 126 10
comb(9,4)*comb(5,4) 126 5
None
comb(9,5),comb(4,4) 126 1
comb(9,5)*comb(4,5) 126 0
None
comb(9,6),comb(3,5) 84 0
comb(9,6)*comb(3,6) 84 0
None
comb(9,7),comb(2,6) 36 0
comb(9,7)*comb(2,7) 36 0
None
comb(9,8),comb(1,7) 9 0
comb(9,8)*comb(1,8) 9 0
None
comb(9,9),comb(0,8) 1 0


In [99]:
moves = [9,2,3,4]
for sub_moves in itertools.combinations(moves, 3):
    print(np.sum(sub_moves))

14
15
16
9


In [5]:
import numpy as np
import itertools
from math import comb


In [26]:
    def init_value_table(player, quiet=True):
        """
        Init the value table.
        """
        value_table = {}
            
        def init_value(player, X_moves, O_moves):
            state = (tuple(sorted(X_moves)), tuple(sorted(O_moves)))
            if player == 'X':
                if any_n_sum_to_k(X_moves):
                    value_table[state] = 1
                elif any_n_sum_to_k(O_moves):
                    value_table[state] = 0
                elif len(X_moves) + len(O_moves) == 9:
                    value_table[state] = 0
                else:
                    value_table[state] = 0.5

            elif player == 'O':
                if any_n_sum_to_k(X_moves):
                    value_table[state] = 0
                elif any_n_sum_to_k(O_moves):
                    value_table[state] = 1
                elif len(X_moves) + len(O_moves) == 9:
                    value_table[state] = 0
                else:
                    value_table[state] = 0.5

        board = list(range(1, 9 + 1))
        n = len(board)
        # As the init only gets run for once at the beginning and only for 6_000 input, lets be silly here.
        for k in range(1, 5 + 1):
            if not quiet:
                print(
                    print("comb(9,%d)*comb(%d,%d)" % (k - 1, n - (k - 1), k - 1), comb(n, k - 1),
                          comb(n - (k - 1), k - 1)))
            for X_moves in itertools.combinations(board, k - 1):
                for O_moves in itertools.combinations(list(set(board) - set(X_moves)), k - 1):
                    init_value(player, X_moves, O_moves)
                    
            if not quiet:
                print("comb(9,%d),comb(%d,%d)" % (k, n - k, k - 1), comb(n, k), comb(n - k, k - 1))
            for X_moves in itertools.combinations(board, k):
                for O_moves in itertools.combinations(list(set(board) - set(X_moves)), k - 1):
                    init_value(player, X_moves, O_moves)

        return value_table

In [15]:
show_state([[], []])

2  9  4
7  5  3
6  1  8


In [27]:
v = init_value_table('O')
for key in v.keys():
    if v[key] == 1:
        print(key)

((1, 2, 3), (4, 5, 6))
((1, 2, 4), (3, 5, 7))
((1, 2, 5), (3, 4, 8))
((1, 2, 6), (3, 4, 8))
((1, 2, 6), (3, 5, 7))
((1, 2, 7), (3, 4, 8))
((1, 2, 7), (4, 5, 6))
((1, 2, 8), (3, 5, 7))
((1, 2, 8), (4, 5, 6))
((1, 2, 9), (3, 4, 8))
((1, 2, 9), (3, 5, 7))
((1, 2, 9), (4, 5, 6))
((1, 3, 4), (2, 5, 8))
((1, 3, 4), (2, 6, 7))
((1, 3, 5), (2, 4, 9))
((1, 3, 5), (2, 6, 7))
((1, 3, 6), (2, 4, 9))
((1, 3, 6), (2, 5, 8))
((1, 3, 7), (2, 4, 9))
((1, 3, 7), (2, 5, 8))
((1, 3, 7), (4, 5, 6))
((1, 3, 8), (2, 4, 9))
((1, 3, 8), (2, 6, 7))
((1, 3, 8), (4, 5, 6))
((1, 3, 9), (2, 5, 8))
((1, 3, 9), (2, 6, 7))
((1, 3, 9), (4, 5, 6))
((1, 4, 5), (2, 6, 7))
((1, 4, 6), (2, 5, 8))
((1, 4, 6), (3, 5, 7))
((1, 4, 7), (2, 5, 8))
((1, 4, 8), (2, 6, 7))
((1, 4, 8), (3, 5, 7))
((1, 4, 9), (2, 5, 8))
((1, 4, 9), (2, 6, 7))
((1, 4, 9), (3, 5, 7))
((1, 5, 6), (2, 4, 9))
((1, 5, 6), (3, 4, 8))
((1, 5, 7), (2, 4, 9))
((1, 5, 7), (3, 4, 8))
((1, 5, 8), (2, 4, 9))
((1, 5, 8), (2, 6, 7))
((1, 6, 7), (2, 4, 9))
((1, 6, 7),

In [30]:
a = [1,1,-1,0,0.5]
a.index(3)

ValueError: 3 is not in list

## Q2

### (e)

### (f)

In [ ]:
n_arms = 10
A_set = list(range(n_arms))

def action_value(t: int, action: int):
    """
    return the turn action value of action a at time t. 
    """
    pass

In [6]:
np.random.seed(1)
np.random.normal(size=(10,1))[1,0]

-0.6117564136500754

In [8]:
np.random.seed(1)
np.cumsum(np.random.normal(loc=0.0, scale=1.0, size=(2,10)), axis=1)

array([[ 1.62434536,  1.01258895,  0.4844172 , -0.58855142,  0.2768562 ,
        -2.02468249, -0.27987073, -1.04107763, -0.72203853, -0.97140891],
       [ 1.46210794, -0.59803277, -0.92044998, -1.30450433, -0.17073489,
        -1.27062616, -1.44305436, -2.32091278, -2.27869903, -1.69588382]])

In [10]:
a = np.random.normal(loc=0.0, scale=1.0)

In [11]:
a

1.1447237098396141

In [12]:
a[1] = 1

TypeError: 'float' object does not support item assignment

In [18]:
estimated_action_value_dict = {
    'a': 1,
    'b': 2,
    'c': 2
}
max(estimated_action_value_dict)

'c'

In [15]:
a = np.array([1,2,3,1])

In [45]:
a * (a == 2)

array([0, 2, 0, 0])

In [2]:
[0] * 10

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [3]:
a = [0]*10

In [4]:
a[1]+=1

In [5]:
a

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [6]:
max(a)

1

In [9]:
if 1:
    print(1)

1


In [8]:
np.random.binomial(1,1)

1

In [14]:
a[[1,2]]

TypeError: list indices must be integers or slices, not list

In [15]:
import operator

In [23]:
get135 = operator.itemgetter((0, 2, 4))

In [24]:
type(get135)

operator.itemgetter

In [25]:
get135(a)

TypeError: list indices must be integers or slices, not tuple

In [26]:
a

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
1 in [1,2,3,]

True

In [20]:
np.argmax([1,3,1])

1

In [28]:
a = np.array([
    [3,2],
    [2,4],
    [1,5],
])
a.take_along_axis(axis=1)

AttributeError: 'numpy.ndarray' object has no attribute 'take_along_axis'

In [26]:
a[:,0]

array([1, 1, 1])

In [30]:
a.argmax(axis=0)

array([0, 2], dtype=int64)

In [26]:
"""
n - number of bandit
action = 0,1,2,3,4,5,6,7,8
epsilon = 0.1
Q* - The real value of action (10, 2000)
Q - The estimated value for each action (10,)
N_a - The # of action i (10,)
randomness
max_num_tasks = 2000
"""
import numpy as np

max_num_tasks = 2000

# Set up
n = 10

# Q = np.zeros(n)  # The estimated value for each action
# N_a = np.zeros(n)
Q_star = np.random.normal(loc=0, scale=1, size=(n, max_num_tasks))  # The real reward
randomness = np.random.normal(max_num_tasks)


def learn(a, r):
    N_a[a] += 1
    Q[a] += (r - Q[a]) / N_a[a]


def reward(a, task_num):
    """
    There is a random walk for each reward.
    """
    return np.random.normal(loc=Q_star[a, task_num], scale=1)


def greedy():
    return np.argmax(Q)


def epsilon_greedy(epsilon):
    if np.random.binomial(n=1, p=epsilon):
        action = np.random.choice(n)  # randomly select one of the 10 beds
    else:
        action = greedy()

    return action


def max_Q_star():
    return Q_star.argmax(axis=0)


def run(num_runs=1000, num_steps=100, epsilon=0):
    avg_reward = np.zeros(num_steps)
    prob_a_star = np.zeros(num_steps)
    optimal_action = max_Q_star()
    for run_num in range(num_runs):  # At game run_num
        a_star = optimal_action[run_num]
        
        Q = np.zeros(n)  # The estimated value for each action
        N_a = np.zeros(n)

        for time_step in range(num_steps):
            a = epsilon_greedy(epsilon)
            r = reward(a, run_num)
            learn(a, r)
            avg_reward[time_step] += r
            if a_star == a:
                prob_a_star[time_step] += 1

        for i in range(num_steps):
            avg_reward[i] = avg_reward[i] / num_runs
            prob_a_star[i] = prob_a_star[i] / num_runs
        
    return avg_reward, prob_a_star

run()

(array([ 2.99067638e-04, -6.71949215e-04, -1.28069884e-05,  1.55223032e-03,
         2.12490543e-04,  8.20312222e-04,  5.01888820e-04,  8.39554543e-04,
         4.14998969e-04,  1.04946738e-03, -6.55769306e-04, -4.13632519e-05,
        -3.34280274e-04,  3.84911903e-04,  1.06744867e-03, -1.29431192e-03,
        -9.61101607e-04, -3.26913248e-04, -4.65067570e-04, -1.52909307e-03,
         4.68166013e-05,  1.50430260e-03, -2.33608861e-04, -2.80045064e-04,
        -6.99426575e-04, -2.61056899e-06, -5.75807857e-04, -3.98592768e-04,
         6.23940206e-04,  1.46049211e-03, -1.04486439e-03, -5.22623794e-04,
         2.61136094e-04, -4.71664996e-04, -1.08138369e-04, -1.46822804e-03,
        -3.18757781e-04,  3.49942502e-04, -7.53300210e-05, -9.20109630e-04,
        -1.00088997e-03, -5.94751466e-04, -1.07639595e-03, -1.12301677e-04,
        -6.85408487e-04,  9.15893599e-04, -1.55219491e-04,  2.77604695e-04,
         1.13336788e-03,  3.21199839e-04, -1.90158068e-03,  1.52902602e-03,
        -2.2

In [25]:
Q

array([ 0.01524362, -1.66640739, -0.57245844, -0.21553452, -0.35449449,
       -0.08003133, -0.32250277, -0.6523634 , -1.01532296, -0.01326089])

In [18]:
num_runs=1000
num_steps=100
epsilon=0

avg_reward = np.zeros(num_steps)
prob_a_star = np.zeros(num_steps)

for run_num in range(num_runs):
    a_star = 0
    n_star = 0
    # Find the true optimal action for each run_num.
    for a in range(1, n):
        if Q_star[a][run_num] > Q_star[a_star][run_num]:
            a_star = a
            n_star = run_num
    
    collect = []
    for time_step in range(num_steps):
        a = epsilon_greedy(epsilon)
        r = reward(a, run_num)
        learn(a, r)
        avg_reward[time_step] += r
        if a_star == a:
            prob_a_star[time_step] += 1

for i in range(num_steps):
    avg_reward[i] = avg_reward[i] / num_runs
    prob_a_star[i] = prob_a_star[i] / num_runs
        
print(a_star, n_star)
Q_star.argmax(axis=0)

6 999


array([4, 6, 5, ..., 7, 2, 4], dtype=int64)

In [19]:
avg_reward

array([ 5.14528501e-03,  3.23724476e-02,  2.04697769e-02,  3.78350818e-02,
        2.86758614e-02,  2.22460366e-02,  1.14760547e-02,  3.55012580e-02,
        3.71913468e-03, -1.10721183e-02,  1.77596322e-02,  1.05402487e-02,
        2.73232032e-02,  4.25418386e-02,  6.05780059e-02,  4.93658654e-02,
        3.28446586e-03,  3.34374097e-02,  1.80636504e-02,  1.36347242e-02,
        4.14024323e-02,  1.20733587e-02, -2.56722102e-02,  2.94638758e-03,
        2.80225114e-02,  1.90232083e-02, -2.23215894e-02,  2.01889211e-02,
        3.84550365e-02,  1.21806187e-02, -5.06335772e-02,  3.63112538e-02,
        1.04770805e-02,  5.77980129e-03,  5.12473235e-03, -1.66182534e-02,
       -3.64647839e-02, -4.08594908e-02,  1.84644592e-02, -1.00240397e-02,
       -9.43205047e-03, -2.76346530e-02,  9.37473981e-03, -5.92402251e-03,
       -2.73498084e-02,  3.86882259e-02,  1.71466623e-02,  6.04128082e-03,
        3.65277849e-02,  7.88193024e-02, -5.11846360e-02,  7.16948061e-02,
        9.66056998e-03, -

In [20]:
prob_a_star

array([0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091, 0.091,
       0.091])